# Simulation for Mori-Zwanzig formalism (deterministic)

 We implement a Langevin integrator with WCA potential to model the dynamics of a bath of particles together with a distinguished particle without any noise (Actually Hamilton equations using an ABOBA algorithm without noise)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.visualization as msmrdvis
from msmrd2.integrators import integratorMoriZwanzigDeterministic 
from msmrd2.potentials import WCA, harmonic
import msmrd2.tools.particleTools as particleTools

## Units

### Boltzman constant
- $k_B = 1.38064852 \times 10^{-23} \frac{m^2}{s^2} \frac{kg}{K} \left(= \frac{nm^2}{ns^2}\frac{kg}{K}\right)$.

### Basic units
- Length (l): nanometers (nm)
- Energy ($\epsilon$) : $k_B T = (g/mol) \frac{nm^2}{ns^2}=1$
- Mass (m): gram/mol (g/mol)

### Derived units
- Time: $l\sqrt{m/\epsilon} = nm$
- Temperature: $\epsilon/k_B =$ Kelvin ($K$)
- Force: $\epsilon/l = kg \frac{nm}{ns^2}$

### Reduced quantities (dimensionless)
- Reduced pair potential: $U^* = U/\epsilon$
- Reduced force: $F^* = F l/\epsilon$
- Reduced distance: $r^* = r/l$
- Reduced density: $\rho^*=\rho l^3$
- Reduced Temperature: $T^* = k_B T/\epsilon$
- Reduced Pressure: $P^* = Pl^3/\epsilon$

In [ ]:
# Main parameters
numBathParticles = 500 #500
numparticles = 1 + numBathParticles #Added distinguished particle (index 0)
Gamma = 0.3 #30 # Friction coefficient (units of KbT/D = mass over time (gram/mol)/ns)
particlemass = 18.0 # (g/mol) approximately mass of H20
distinguishedParticleMass = 3 * particlemass # (kg)
distinguishedTypes = [1]

numSimulations = 500 #250 #500
boxsize = 8
particleDiameter = 0.3 #nm
separationDistance = 2*particleDiameter # minimum separation distance for initial condition
# For computations, we assume KbT=1, thus the force F must be: F=KbT f, where f is the force computed
# from the potential. This means the plotted potential is on reduced units (not the distances though);
KbT = 0.01
sigmavel=2*KbT*Gamma
seed = -1 # Seed = -1 used random device as seed

# Define arbitrarily large list of particles (velocities assumed zero at initial position)
partlist = particleTools.randomLangevinParticleVelocityList(numparticles, boxsize, separationDistance,sigmavel,
                                                        particlemass, seed, distinguishedParticleOrigin=False)
# Set distinguished particle (default type is zero)
partlist[0].setType(1)
partlist[0].setMass(distinguishedParticleMass)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.05 #0.2
seed = -1 # Seed = -1 used random device as seed
bodytype = 'point'
integrator = integratorMoriZwanzigDeterministic(dt, seed, bodytype, Gamma) 
integrator.setKbT(KbT)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
#boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)
integrator.setDistinguishedTypes(distinguishedTypes)

In [ ]:
# Define WCA potential (rm=2^(1/6)sigma)
epsilon = 1.0
rm = particleDiameter
sigma = rm * 2**(-1/6)
forceCapVal = 100.0
potentialWCA = WCA(epsilon, sigma)
potentialWCA.setForceCapValue(forceCapVal)
integrator.setAuxPairPotential(potentialWCA) # Aux so it can be saved into aux variable

In [ ]:
# Define external potential
minima = np.array([0.,0.,0.])
kconstant = np.array([0.3,0.3,0.3])
partTypes = [1] # Only acts on particle types one
scalefactor = 1
externalPotential = harmonic(minima, kconstant, partTypes, scalefactor)
integrator.setExternalPotential(externalPotential)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 2000
stride = 5
datafile  = open('../../../data/vmd/MoriZwanzigDeterministic.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            if part.type == 0:
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            else:
                datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
    integrator.integrate(partlist)
    if i%1000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
print("Percentage complete: ", 100, " %")

In [ ]:
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_MoriZwanzig(numparticles = numparticles, outfname = "MoriZwanzigDeterministic", 
                              tclfname = "../../../data/vmd/MoriZwanzigDeterministic_2vmd.tcl", 
                                 particleDiameter = particleDiameter)

To load the movie go to /data/vmd and run in a terminal "vmd -e langevinLennardJones_2vmd.tcl".


## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions (prints positions and orientations of both particles)
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Positions", "Velocites"))
for i in range(timesteps):
    print('{:<10d}{:<10f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[0].velocity)))
    print('{:<10s}{:<10s}{:<40s}{:<50s}'.format(str(" "), str(" "), str(partlist[1].position), str(partlist[1].velocity)))
    integrator.integrate(partlist)